In [9]:
import sys
sys.path.append('../../code/')

In [10]:
from tqdm.auto import tqdm
tqdm.pandas()

/home/espinl/.conda/envs/py311_llmscholar/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
%load_ext autoreload
%autoreload 2

from libs import io
from libs import constants
from libs.network import fragmentation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
FN_COAUTHORSHIP = '../../../APS/results/augmented_aps/coauthorships.csv'
FACTUALITY_PATH = '../../results/interventions/factuality'
RESULTS_PATH = '../../results/interventions/tables'

In [13]:
# Load the summary data for each model (all outputs)
df_factuality_author_all = io.pd.concat([io.read_csv(io.path_join(FACTUALITY_PATH, f"{model}_author.csv"), index_col=0, low_memory=False) for model in constants.LLMS], ignore_index=True)
df_factuality_author_all.shape


(3365532, 36)

In [14]:
# filter period (valid for open-weight models)
# 2026-01-15 some models were not available (eg. gemma, qwen)
start_date = '2025-12-19'
end_date = '2026-01-18'

# filter out interventions
query = "((not model.str.contains('gemini') and date >= @start_date and date <= @end_date) or model.str.contains('gemini'))"

df_factuality_author = df_factuality_author_all.query(query).copy()

# shapes
df_factuality_author.shape


(3196771, 36)

In [21]:
df_coauthorships = io.pd.read_csv(FN_COAUTHORSHIP)
df_coauthorships.shape

(37022942, 2)

In [22]:
oa_ids_in_recommendations = df_factuality_author.id_author_oa.unique()
df_coauthorships_in_recommendations = df_coauthorships.query("src in @oa_ids_in_recommendations and dst in @oa_ids_in_recommendations and src != dst").copy()
df_coauthorships_in_recommendations = df_coauthorships_in_recommendations.drop_duplicates()
df_coauthorships_in_recommendations.shape
# (350554, 2)

(175277, 2)

In [24]:
group_cols = ['model','grounded','temperature','date','time','task_name','task_param','task_attempt']
df_factuality_author.groupby(group_cols).ngroups

31765

In [ ]:
group_cols = ['model','grounded','temperature','date','time','task_name','task_param','task_attempt']
results = []

g = df_factuality_author.groupby(group_cols)

for group, df in tqdm(g, total=g.ngroups, desc="Processing groups"):
 
        rec_ids = df.id_author_oa.dropna().unique()

        connectedness = fragmentation.norm_entropy_R_from_edgelist(rec_ids,
                                                                df_coauthorships_in_recommendations,
                                                                src_col = "src",
                                                                dst_col = "dst")

        obj = {c: group[i] for i, c in enumerate(group_cols)}
        obj |= {'nrecs': connectedness.n,
                'n_components': connectedness.n_components,
                'metric': connectedness.norm_entropy,
                'n_edges_rows': connectedness.n_edges_rows,
                'n_edges_undirected_unique': connectedness.n_edges_undirected_unique
                }
        
        results.append(obj)

        if len(rec_ids) == 10:
                break

df_summary = io.pd.DataFrame(results)
io.save_csv(df_summary, io.path_join(RESULTS_PATH, 'per_attempt_connectedness.csv'))

Processing groups:   0%|          | 19/31765 [00:02<1:15:08,  7.04it/s]

[2026-01-29 17:25:02] Data successfully saved to ../../results/interventions/tables/per_attempt_connectedness.csv


In [30]:
len(results)

20

In [31]:
df_summary.shape



(20, 13)

In [32]:
df_summary

,model,grounded,temperature,date,time,task_name,task_param,task_attempt,nrecs,n_components,metric,n_edges_rows,n_edges_undirected_unique
0,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_citations_high,2,44,43,0.991674,1,1
1,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_diverse,1,92,84,0.970008,8,8
2,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_ethnicity_asian,2,34,34,1.000000,0,0
3,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_ethnicity_black,2,4,4,1.000000,0,0
4,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_ethnicity_equal,1,96,86,0.965974,10,10
5,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_ethnicity_latino,1,17,17,1.000000,0,0
6,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_gender_equal,2,91,78,0.945350,13,13
7,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_gender_female,2,39,38,0.990297,1,1
8,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_gender_male,1,96,74,0.884818,23,23
9,deepseek-chat-v3.1,False,0.0,2025-12-19,08:00,biased_top_k,top_100_bias_gender_neutral,1,93,82,0.950001,11,11


In [38]:
model = 'deepseek-chat-v3.1'
grounded = False
temperature = 0.0
date = '2025-12-19'
time = '08:00'
task_name = 'twins'
task_param = 'famous_male'
task_attempt = 1
tmp = df_factuality_author.query("model == @model and grounded == @grounded and temperature == @temperature and date == @date and time == @time and task_name == @task_name and task_param == @task_param and task_attempt == @task_attempt")
tmp[['id_author_oa','clean_name']]

,id_author_oa,clean_name
3151533,5.000679e+09,duncan j watts
3151534,5.067142e+09,mark e j newman
3151535,5.046547e+09,alessandro vespignani
3151536,5.029419e+09,steven h strogatz
3151537,5.012905e+09,reka albert
3151538,5.111720e+09,m e cates
3151539,5.022368e+09,ginestra bianconi
3151540,5.057078e+09,marta gonzalez
3151541,5.073544e+09,luis a nunes amaral
3151542,5.109845e+09,h eugene stanley


In [39]:
df_coauthorships.query("src in @tmp.id_author_oa and dst in @tmp.id_author_oa")

,src,dst
1325742,5073544378,5109845199
5374550,5029418921,5067142016
9543269,5000679279,5067142016
11196440,5029418921,5067142016
11806910,5000679279,5029418921
11806912,5000679279,5067142016
11806914,5029418921,5067142016
21808272,5000679279,5029418921
21808273,5000679279,5067142016
21808274,5029418921,5067142016


___


In [1]:
import sys
sys.path.append('../../code/')

In [2]:
%load_ext autoreload
%autoreload 2

from libs import io
from libs import constants

Available LLMs: (24): llama-3.3-8b llama-4-scout llama-4-mav gpt-oss-20b gpt-oss-120b qwen3-8b qwen3-14b qwen3-32b qwen3-30b-a3b-2507 qwen3-235b-a22b-2507 gemma-3-12b gemma-3-27b mistral-small-3.2-24b mistral-medium-3 llama-3.1-70b llama-3.3-70b llama-3.1-405b grok-4-fast deepseek-chat-v3.1 deepseek-r1-0528 gemini-2.5-flash gemini-2.5-flash-grounded gemini-2.5-pro gemini-2.5-pro-grounded


,id_publication,id_author,position,id_institution,authorship_flag
0,2155512932,5061426463,first,4.210121e+09,original_openalex
1,2155512932,5064510725,middle,1.282106e+09,original_openalex
2,2155512932,5111938223,middle,4.210121e+09,original_openalex
3,2155512932,5052336272,middle,4.210121e+09,original_openalex
4,2155512932,5110594767,middle,4.210121e+09,original_openalex


In [8]:
float('nan') == None

False